In [ ]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt

# Importa o módulo fuzzy_ode do local correto
from fuzzy_systems.dynamics.fuzzy_ode import FuzzyODESolver, FuzzyNumber

# ============================================================================
# MODELO LOGÍSTICO
# ============================================================================

def logistic_model(t, y, r, K):
    """
    Modelo logístico: dy/dt = r * y * (1 - y/K)

    Argumentos:
        t: Tempo (não usado)
        y: Vetor de estado [y]
        r: Taxa de crescimento
        K: Capacidade suporte (carrying capacity)

    Retorna:
        dy/dt
    """
    return r * y * (1 - y / K)

In [ ]:
# ========== CENÁRIO A: Condição Inicial Fuzzy ==========
t_span = (0, 50)      # Tempo: 0 a 50
r_base = 0.2       # Taxa de crescimento
K_base = 100  

y0_fuzzy_a = FuzzyNumber.triangular(center=10, spread=4)

solver_a = FuzzyODESolver(
    ode_func=logistic_model,
    t_span=t_span,
    initial_condition=[y0_fuzzy_a],
    params={'r': r_base, 'K': K_base},
    n_alpha_cuts=25,
)

sol_a = solver_a.solve(method='monte_carlo',verbose=True)
# sol_a = solver_a.solve(method='standard',n_grid_points=100,verbose=True)
sol_a.plot()

In [ ]:
K_fuzzy_b = FuzzyNumber.triangular(center=500, spread=100)

solver_b = FuzzyODESolver(
    ode_func=logistic_model,
    t_span=t_span,
    initial_condition=[10.0],
    params={'r': r_base, 'K': K_fuzzy_b},
    n_alpha_cuts=25,
)

sol_b = solver_b.solve(method='monte_carlo')
sol_b.plot()

In [ ]:

r_fuzzy_c = FuzzyNumber.triangular(center=0.2, spread=0.02)

solver_c = FuzzyODESolver(
    ode_func=logistic_model,
    t_span=t_span,
    initial_condition=[10.0],
    params={'r': r_fuzzy_c, 'K': K_base},
    n_alpha_cuts=25,
)

sol_c = solver_c.solve(method='monte_carlo',verbose=True)
sol_c.plot()


In [ ]:
y0_fuzzy_d = FuzzyNumber.triangular(center=10, spread=4)
r_fuzzy_d = FuzzyNumber.triangular(center=0.2, spread=0.02)
K_fuzzy_d = FuzzyNumber.triangular(center=500, spread=100)

solver_d = FuzzyODESolver(
    ode_func=logistic_model,
    t_span=t_span,
    initial_condition=[y0_fuzzy_d],
    params={'r': r_fuzzy_d, 'K': K_fuzzy_d},
    n_alpha_cuts=25,
)

sol_d = solver_d.solve(method='monte_carlo',verbose=True,n_samples=10000)
sol_d.plot()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

scenarios = [
    (sol_a, 'Cenário A: y₀ Fuzzy'),
    (sol_b, 'Cenário B: K Fuzzy'),
    (sol_c, 'Cenário C: r Fuzzy'),
    (sol_d, 'Cenário D: Todos Fuzzy')
]

for ax, (sol, title) in zip(axes, scenarios):
    cmap = plt.cm.Blues
    for i, alpha in enumerate(sol.alphas):
        y_min = sol.y_min[i, 0, :]
        y_max = sol.y_max[i, 0, :]
        color = cmap(0.3 + 0.7 * alpha)
        ax.fill_between(sol.t, y_min, y_max, alpha=0.3, color=color)

    ax.set_xlabel('Tempo (t)', fontsize=11)
    ax.set_ylabel('y(t)', fontsize=11)
    ax.set_title(title, fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
# plt.savefig('fuzzy_ode_logistic_scenarios.png', dpi=150, bbox_inches='tight')
